In [ ]:
# importing packages 
import pprint
import requests
import xml.etree.ElementTree as ET
from collections import defaultdict
pp = pprint.PrettyPrinter(indent=2)


In [24]:
# API url
url = 'http://mediafeed.boostmotorgroup.com/2438/export.xml'
  
# creating HTTP response object from given url
response = requests.get(url)

# parsing XML
root = ET.fromstring(response.content)

# from XML to dictionary
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

dict_data = etree_to_dict(root)

# store common dealership data
dealership_data = dict_data['Datafeed']['Dealership']
dealership = {
    'Dealership_Boost_ID': dealership_data['Dealership_Boost_ID'],
    'Dealership_Other_ID': dealership_data['Dealership_Other_ID'],
    'Dealership_Name': dealership_data['Dealership_Name'],
    'Dealership_Website': dealership_data['Dealership_Website'],
    'Dealership_Phone': dealership_data['Dealership_Phone'],
    'Dealership_Email': dealership_data['Dealership_Email'],
    'Dealership_Address': dealership_data['Dealership_Address'],
    'Dealership_City': dealership_data['Dealership_City'],
    'Dealership_Postal': dealership_data['Dealership_Postal'],
    'Dealership_Province': dealership_data['Dealership_Province'],
    'Dealership_Latitude': dealership_data['Dealership_Latitude'],
    'Dealership_Longitude': dealership_data['Dealership_Longitude'],
}

vehicles = []
all_vehicles = dict_data['Datafeed']['Dealership']['Inventory']['Vehicle']
# loop over all vehicles, store each unique vehicle and attach dealership data to it
for v in all_vehicles:
    vehicle_info = {}
    features = []
    for key, value in v.items():
        vehicle_info[key] = value
    vehicle = {**dealership, **vehicle_info}
    # use the first image as the main one
    imageGallery = []
    main_img_url = 'https://media.getedealer.com/w_400,h_300,q_90,c_l,v1/inventory/6RHU7CPFWRG7TFNE6JCXOPN42Y.webp'
    if vehicle['Images']['@count'] == '1':
        main_img_url = vehicle['Images']['Photo']['#text']
    elif not vehicle['Images']['@count'] == '0':
        # use the next maximum of 25 images in the multi-image field
        main_img_url = vehicle['Images']['Photo'][0]['#text']
        images = vehicle['Images']['Photo'][1:]
        i = 0
        for image in images:
            if i == 25: break
            imageGallery.append({'url': image['#text']})
            i+=1
    vehicle['Main_Image'] = {"url": main_img_url}
    vehicle['Images'] = imageGallery
    # store the other multivalues into strings
    vehicle['Features'] = str(vehicle['Features'])
    vehicle['Interior_Features'] = str(vehicle['Interior_Features'])
    vehicle['Exterior_Features'] = str(vehicle['Exterior_Features'])
    vehicle['Safety_Features'] = str(vehicle['Safety_Features'])
    key_list = ["_draft", "_archived", "price2", "price", "seats", "doors", "mileage", "year", "vehiclestatus", "special", "certified", "features", "safety-features", "exterior-features", "interior-features", "interior-colour", "exterior-colour", "body-style", "submodel-trim", "dealership-name", "main-image", "images", "transmission", "model", "make", "name", "dealership-boost-id"]
    vehicle =  {k.lower().replace('_','-'): v for k, v in vehicle.items()}
    
    vehicle['name'] = vehicle['vin']
    vehicle['_archived'] = False
    vehicle['_draft'] = False
    vehicle = {k: v for k, v in vehicle.items() if k in key_list}

    vehicles.append(vehicle)

vehicles

[{'dealership-boost-id': '6429',
  'dealership-name': 'Carmoji.ca',
  'year': '2022',
  'make': 'Ram',
  'model': '1500',
  'submodel-trim': 'Laramie 4x4 Crew Cab 5&apos;7&quot; Box',
  'body-style': 'Pickup Truck',
  'exterior-colour': 'Black',
  'interior-colour': 'Black',
  'mileage': '6563',
  'doors': '4',
  'seats': '5',
  'transmission': 'Automatic',
  'price': '69987',
  'price2': '69987',
  'interior-features': '3,17,24,28,44,669,844,1154,1595,2119,3451,255541',
  'exterior-features': '99,1557',
  'safety-features': '722,3335',
  'certified': 'false',
  'special': 'false',
  'vehiclestatus': 'Used',
  'images': [{'url': 'https://media.getedealer.com/w_400,h_300,q_90,c_l,v1/inventory/6RHU7CPFWRG7TFNE6JCXOPN42Y.webp'},
   {'url': 'https://media.getedealer.com/w_400,h_300,q_90,c_l,v1/inventory/6RHU7CPFWRG7TFNE6JCXOPN42Y.webp'},
   {'url': 'https://media.getedealer.com/w_400,h_300,q_90,c_l,v1/inventory/6RHU7CPFWRG7TFNE6JCXOPN42Y.webp'},
   {'url': 'https://media.getedealer.com/w_4

In [25]:
WEBFLOW_API_KEY="3588c873fdc9a472583c06d2e04973049d787e20e7fa54e6485fad715ce4753f"
WEBFLOW_SITE_ID="64a57674f0a3977d20e3865b"
WEBFLOW_COLLECTION_ID="64aaff5935018faa5a99900f"

CONFIG = {
    "Authorization": f"Bearer {WEBFLOW_API_KEY}",
    "accept-version": "1.0.0",
    "content-type": "application/json",
}

url = f"https://api.webflow.com/collections/{WEBFLOW_COLLECTION_ID}/items"
for vehicle in vehicles:
    data = { "fields": vehicle}
    response = requests.post(url, json=data, headers=CONFIG)
    print("Status Code", response.status_code)
    print("JSON Response ", response.json())
    break

Status Code 200
JSON Response  {'_archived': False, '_draft': False, 'year': 2022, 'mileage': 6563, 'doors': 4, 'seats': 5, 'price': 69987, 'price2': 69987, 'dealership-boost-id': '6429', 'dealership-name': 'Carmoji.ca', 'make': 'Ram', 'model': '1500', 'submodel-trim': 'Laramie 4x4 Crew Cab 5&apos;7&quot; Box', 'body-style': 'Pickup Truck', 'exterior-colour': 'Black', 'interior-colour': 'Black', 'transmission': 'Automatic', 'interior-features': '3,17,24,28,44,669,844,1154,1595,2119,3451,255541', 'exterior-features': '99,1557', 'safety-features': '722,3335', 'certified': 'false', 'special': 'false', 'vehiclestatus': 'Used', 'features': "{'Feature': [{'@category': 'Interior', '#text': 'Air Conditioning'}, {'@category': 'Interior', '#text': 'Cruise Control'}, {'@category': 'Interior', '#text': 'Heated Seats'}, {'@category': 'Interior', '#text': 'Keyless Entry'}, {'@category': 'Interior', '#text': 'Power Windows'}, {'@category': 'Exterior', '#text': 'Alloy Wheels'}, {'@category': 'Interior

In [27]:
img_data = requests.get('https://boostcarimages.blob.core.windows.net/carimages/6429/5907238/2022%20Ram%201500%20638237759080367720_1.jpg').content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)

In [ ]:
files = {'upload_file': open('file.txt','rb')}